<a href="https://colab.research.google.com/github/vaishna-v/DMBI/blob/main/exp_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from plotly.offline import init_notebook_mode, iplot_mpl, download_plotlyjs, plot, iplot
import plotly.figure_factory as ff
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)
import statsmodels.formula.api as sm
import missingno as msno
from sklearn.preprocessing import LabelEncoder
from statsmodels.compat import lzip
import statsmodels.api as sm
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
try:
    import apyori
except:
    !pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5976 sha256=fc6a6a6ddefe488f4e1ba52df857fc6ff07736778a576ab19db8f904648ebf95
  Stored in directory: /root/.cache/pip/wheels/32/2a/54/10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori


In [ ]:
from apyori import apriori


In [ ]:
df=pd.read_csv("lastfm.csv")
df

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289950,19718,bob dylan,f,Canada
289951,19718,pixies,f,Canada
289952,19718,the clash,f,Canada
289953,19718,a tribe called quest,f,Canada


In [ ]:
df.shape


(289955, 4)

In [ ]:
df.describe()

,user
count,289955.000000
mean,9852.460447
std,5692.355041
min,1.000000
25%,4935.000000
50%,9838.000000
75%,14769.000000
max,19718.000000


In [ ]:
df[df.duplicated(keep=False)]


,user,artist,sex,country
102441,6980,m.i.a.,f,United Kingdom
102446,6980,m.i.a.,f,United Kingdom
143737,9753,james brown,m,Germany
143746,9753,james brown,m,Germany


In [ ]:
df.drop_duplicates(inplace=True)


In [ ]:
print("User unique: ",len(df.user.unique()))
print("Artist unique: ",len(df.artist.unique()))
print("Country unique: ",len(df.country.unique()))

User unique:  3698
Artist unique:  1004
Country unique:  106


In [ ]:
len(df.artist.unique())


1004

In [ ]:
df = df[['user','artist']]


In [ ]:
transactions = []
for i in df['user'].unique():
    transactions.append(list(df[df['user'] == i]['artist'].values))

In [ ]:
len(transactions)


3698

In [ ]:
# We check the values of the first 5 users in the list
transactions[0:5]

[['red hot chili peppers',
  'the black dahlia murder',
  'goldfrapp',
  'dropkick murphys',
  'le tigre',
  'schandmaul',
  'edguy',
  'jack johnson',
  'eluveitie',
  'the killers',
  'judas priest',
  'rob zombie',
  'john mayer',
  'the who',
  'guano apes',
  'the rolling stones'],
 ['devendra banhart',
  'boards of canada',
  'cocorosie',
  'aphex twin',
  'animal collective',
  'atmosphere',
  'joanna newsom',
  'air',
  'portishead',
  'massive attack',
  'broken social scene',
  'arcade fire',
  'plaid',
  'prefuse 73',
  'm83',
  'the flashbulb',
  'pavement',
  'goldfrapp',
  'amon tobin',
  'sage francis',
  'four tet',
  'max richter',
  'autechre',
  'radiohead',
  'neutral milk hotel',
  'beastie boys',
  'aesop rock',
  'mf doom',
  'the books'],
 ['tv on the radio',
  'tool',
  'kyuss',
  'dj shadow',
  'air',
  'a tribe called quest',
  'the cinematic orchestra',
  'beck',
  'bon iver',
  'röyksopp',
  'bonobo',
  'the decemberists',
  'snow patrol',
  'battles',
  't

In [ ]:
association_rules = apriori(transactions, min_support=0.03, min_confidence=0.4, min_lift=2)
association_results = list(association_rules)

In [ ]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))


Apriori = pd.DataFrame(inspect(association_results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

Apriori['Transaction']= Apriori['Left Hand Side']+"--"+Apriori['Right Hand Side']
Apriori.sort_values(by='Lift',inplace=True,ascending=False)

In [ ]:
Apriori

,Left Hand Side,Right Hand Side,Support,Confidence,Lift,Transaction
4,led zeppelin,pink floyd,0.037047,0.434921,4.041047,led zeppelin--pink floyd
10,the rolling stones,the beatles,0.034072,0.512195,2.913996,the rolling stones--the beatles
1,bob dylan,the beatles,0.040022,0.505119,2.873741,bob dylan--the beatles
9,the smashing pumpkins,radiohead,0.031639,0.502146,2.788192,the smashing pumpkins--radiohead
2,death cab for cutie,coldplay,0.035425,0.401840,2.682322,death cab for cutie--coldplay
3,david bowie,the beatles,0.036236,0.452703,2.575530,david bowie--the beatles
7,sigur rós,radiohead,0.030287,0.455285,2.527991,sigur rós--radiohead
6,queen,the beatles,0.032720,0.440000,2.503262,queen--the beatles
5,led zeppelin,the beatles,0.035154,0.412698,2.347937,led zeppelin--the beatles
0,air,radiohead,0.030557,0.410909,2.281594,air--radiohead
